In [ ]:
# !pip3 install holidays

In [1]:
import warnings
import pandas as pd
import datetime
import numpy as np

In [ ]:
# df = pd.read_csv('/home/faye/data/vehicles_2018_cleaned_dataset')

In [ ]:
# Converting date columns to datetime
# datetime_columns = ['DAYOFSERVICE', 'LASTUPDATE']
# for column in datetime_columns:
#     df[column] = pd.to_datetime(df[column])

In [ ]:
# Getting the Irish holidays - including bank holidays
# https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
# irish_holidays_2018 = []
# for date in holidays.Ireland(years=2018).items():
#     irish_holidays_2018.append(str(date[0]))

In [ ]:
# irish_holidays_2018

In [ ]:
# df['Is_HOLIDAY'] = [1 if str(val).split()[0] in irish_holidays_2018 else 0 for val in df['DAYOFSERVICE']]
# df['IS_WEEKDAY'] = [1 if int(val.weekday()) < 5 else 0 for val in df['DAYOFSERVICE']]
# df['DAYOFWEEK'] = [calendar.day_name[val.weekday()] for val in df['DAYOFSERVICE']]
# df['MONTHOFSERVICE'] = df['DAYOFSERVICE'].dt.strftime('%B')

In [2]:
leavetimes = pd.read_feather('/home/faye/data/leavetimes_3_extra_features.feather')

In [3]:
leavetimes = leavetimes.sort_values(by=['TRIPID', 'DAYOFSERVICE', 'PROGRNUMBER'])

In [ ]:
leavetimes.head(50)

In [4]:
# New feature - actualtime between stops
leavetimes['ACTUALTIME_BETWN_STOPS'] = (leavetimes['ACTUALTIME_ARR']-leavetimes['ACTUALTIME_DEP'].shift()).fillna(0)

In [ ]:
leavetimes.tail(50)

In [ ]:
leavetimes.to_csv('/home/faye/data/leavetimes_3_extra_features', index=True)

In [5]:
leavetimes.reset_index(level=0, inplace=True)

In [6]:
leavetimes.head(5)

,index,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,VEHICLEID,PLANNEDTIME_ARR,ACTUALTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_DEP,MONTHOFSERVICE,DAYOFWEEK,DWELLTIME,ACTUALTIME_BETWN_STOPS
0,1590303,2018-01-06,5955221,1,248,2693254,37800,37836,37800,37836,January,Saturday,0,0.0
1,9809509,2018-01-06,5955221,2,249,2693254,37857,37875,37857,37904,January,Saturday,29,39.0
2,1590364,2018-01-06,5955221,3,250,2693254,37962,37956,37962,37972,January,Saturday,16,52.0
3,1591543,2018-01-06,5955221,4,251,2693254,38023,37993,38023,38006,January,Saturday,13,21.0
4,1591575,2018-01-06,5955221,5,252,2693254,38097,38056,38097,38068,January,Saturday,12,50.0


In [7]:
# df = pd.read_csv('/home/faye/data/leavetimes_3_extra_features')
leavetimes.to_feather('/home/faye/data/leavetimes_3_extra_features.feather')

# Trying it in Dask

In [ ]:
leavetimes = dd.read_csv('/home/faye/data/leavetimes_cleaned_2.csv')

### I want to find a way to implement what I am trying to do using Dask dataframes

In [ ]:
leavetimes = leavetimes.set_index('Unnamed: 0')

In [ ]:
# Convert to date times
leavetimes['DAYOFSERVICE'] = leavetimes['DAYOFSERVICE'].astype("M8[us]")
# Convert to epoch

In [ ]:
# Make a list of all of the unique TRIPID and DAYOFSERVICE
tripid = [list(leavetimes['TRIPID'].unique()), list(leavetimes['DAYOFSERVICE'].unique())]

In [ ]:
# Make a new feature named TOTALTIME and only put np.nan values
leavetimes['TOTALTIME'] = np.nan

In [ ]:
# Define a function that will
def time_between(prog_num):
    # return a list comprehension of sliced rows based on the values of PROGRNUMBER's indexes
    return [0 if stop == 1
           else
           (int(row['ACTUALTIME_ARR'].loc[row['PROGRNUMBER']==stop])) - 
            (int(row['ACTUALTIME_DEP'].loc[row['PROGRNUMBER']==(prog_num[prog_num.index(stop)-1])]))
           for stop in prog_num]


In [ ]:
# Loop through the list trips
for id_ in tripid[0]:
    # Loop through list of dates
    for date in tripid[1]:
        # Get rows of matching trips and dates
        row = leavetimes.loc[(leavetimes['TRIPID']==id_) & (leavetimes['DAYOFSERVICE']==date)]
        # Drop TOTALTIME
        row.drop('TOTALTIME', axis=1)
        # Let row totaltime equal to list by calling function
        row['TOTALTIME'] = time_between(list(row['PROGRNUMBER']))
        # Replace the leavetimes rows according to the sub df which has new values for TOTALTIME
        leavetimes.merge(row, left_on='leavetimes', right_index=True)

In [ ]:
b = [1,2,3,4,5,56]
for i in b:
    if b.index(i) == 0:
        print('Here')
    else:
        print(i)